In [1]:


# Standard imports
import argparse
from datetime import datetime
import logging
import os
from pathlib import Path
import sys
from shutil import copyfile
import unittest
import numpy as np

os.chdir('/nas/cee-water/cjgleason/colin/output/')

# Local imports
from output.Append import Append
from output.modules.AbstractModule import AbstractModule
from output.modules.Consensus import Consensus
# from Upload import Upload

In [2]:

def create_args():
    """Create and return argparser with arguments."""

    arg_parser = argparse.ArgumentParser(description="Append results of Confluence workflow execution to the SoS.")
    arg_parser.add_argument("-i",
                            "--index",
                            type=int,
                            help="Index to specify input data to execute on, value of -235 indicates AWS selection")
    arg_parser.add_argument("-c",
                            "--contjson",
                            type=str,
                            help="Name of the continent JSON file",
                            default="continent.json")
    arg_parser.add_argument("-r",
                            "--runtype",
                            type=str,
                            choices=["constrained", "unconstrained"],
                            help="Current run type of workflow: 'constrained' or 'unconstrained'",
                            default="constrained")
    arg_parser.add_argument("-m",
                            "--modules",
                            nargs="+",
                            default=[],
                            help="List of modules executed in current workflow.")
    arg_parser.add_argument("-j",
                            "--metadatajson",
                            type=Path,
                            default=Path(__file__).parent / "metadata" / "metadata.json",
                            help="Path to JSON file that contains global attribute values")
    arg_parser.add_argument("-u",
                            "--podaacupload",
                            action="store_true",
                            help="Indicate requirement to upload to PO.DAAC S3 Bucket")
    arg_parser.add_argument("-b",
                            "--podaacbucket",
                            type=str,
                            help="Name of PO.DAAC S3 bucket to upload to")
    arg_parser.add_argument("-s",
                            "--sosbucket",
                            type=str,
                            default="confluence-sos",
                            help="Name of SoS S3 bucket to upload to")
    return arg_parser


In [3]:
def get_logger():
    """Return a formatted logger object."""
    
    # Create a Logger object and set log level
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)

    # Create a handler to console and set level
    console_handler = logging.StreamHandler()

    # Create a formatter and add it to the handler
    console_format = logging.Formatter("%(asctime)s - %(module)s - %(levelname)s : %(message)s")
    console_handler.setFormatter(console_format)

    # Add handlers to logger
    logger.addHandler(console_handler)

    # Return logger
    return logger

In [4]:
#need a mnt to call on
#/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt
INPUT = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/input")
FLPE = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/flpe")
MOI = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/moi")
DIAGNOSTICS = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/diagnostics")
OFFLINE = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/offline")
VALIDATION = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/validation")

OUTPUT = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/output")
LAKEFLOW = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/flpe/lakeflow")
SSC = Path("/nas/cee-water/cjgleason/ellie/SWOT/confluence/confluence_relPermML/relPermML_mnt/data/flpe/ssc")

#consensus needs to live separately
CONSENSUS = Path('/nas/cee-water/cjgleason/colin/output/tests/flpe')

In [14]:
SOS_NEW = Path('/nas/cee-water/cjgleason/colin/output/tests/sos_new/na_apriori_rivers_v07_SOS_results.nc')
SD_DIR = Path('/nas/cee-water/cjgleason/colin/output/tests/flpe')
SD_SOS = Path('/nas/cee-water/cjgleason/colin/output/tests/flpe/na_apriori_rivers_v07_SOS_results.nc')
FILL = {
    "f8": -999999999999.0,
    "i4": -999,
    "S1": "x",
    "S20": 'xxxxxxxxxxxxxxxxxxxx'
}

def get_sos_data(self):
    """Retrieve and return dictionary of SoS data."""
    ds = Dataset(self, 'r')
    rids = ds["reaches"]["reach_id"][:]
    nrids = ds["nodes"]["reach_id"][:]
    nids = ds["nodes"]["node_id"][:]
    ds.close()
    return { "reaches": rids, "node_reaches": nrids, "nodes": nids }

In [15]:
# File operations to prepare for test
from netCDF4 import Dataset

copyfile(SOS_NEW, SD_SOS)
sos_data = get_sos_data(SD_SOS)

In [16]:
# sos_data

In [17]:
# Logging
logger = get_logger()

# Command line arguments
sd = Consensus([7,8,9], SD_DIR, SOS_NEW, logger, None,None,None, sos_data["reaches"])
sd_dict = sd.get_module_data()

In [18]:
# Create vlen types in SOS
sos = Dataset(SD_SOS, 'a')
vlen_f = sos.createVLType(np.float64, "vlen_float")
vlen_i = sos.createVLType(np.int32, "vlen_int")
vlen_s = sos.createVLType("S1", "vlen_str")
sos.close()

In [19]:
# Run method
sd.append_module_data(sd_dict,'/nas/cee-water/cjgleason/colin/output/metadata/metadata.json')

RuntimeError: NetCDF: String match to name in use: (variable 'consensus_q', group 'consensus')

In [ ]:
# # Append SoS data
# append = Append(INPUT / args.contjson, index, INPUT, OUTPUT, args.modules, \
#     logger, args.metadatajson)
# append.create_new_version()
# # append.create_modules(args.runtype, INPUT, DIAGNOSTICS, FLPE, MOI, OFFLINE, \
# #     VALIDATION / "stats", CONSENSUS, LAKEFLOW, SSC)
# #first, hack to get the consensus to go first
# append.create_modules(args.runtype,INPUT, CONSENSUS)
# append.append_data()
# append.update_time_coverage()